# LangSmith 데이터셋 생성

자체 RAG 평가용 데이터셋을 구축하는 방법에 대해 알아보겠습니다.

먼저, 데이터셋 구축을 위해서는 크게 3가지 과정에 대한 이해가 필요합니다.

Case: Retrieval 이 Question 에 Relevant 한지 평가

> Question - Retrieval

![](./assets/eval-03.png)

Case: Answer 이 Question 에 Relevant 한지 평가

> Question - Answer   

![](./assets/eval-04.png)

Case: Answer 가 Retrieval 된 문서 안에서 답변하였는지 (Hallucination Check)

> Retrieval - Answer

![](./assets/eval-05.png)

따라서, `Question`, `Retrieval`, `Answer` 의 3가지 정보가 필요한 것이 일반적이지만, `Retrieval` 에 대한 Ground Truth 구축이 사실상 어렵습니다.

만약, `Retrieval` 에 대한 Ground Truth 가 존재한다면, 모두 데이터셋으로 저장하여 활용하고, 그렇지 않다면 `Question`, `Answer` 만으로 데이터셋을 구축하여 활용할 수 있습니다.

In [25]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [19]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


데이터셋 생성

inputs 와 outputs 를 활용하여 데이터셋을 생성합니다.

데이터셋은 question 과 answer 로 구성됩니다.



In [20]:
import pandas as pd

# 질문과 답변 목록
inputs = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표한 날은 언제인가요?",
    "코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.",
]

# 질문에 대한 답변 목록
outputs = [
    "삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.",
    "2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.",
    "코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스 상태를 추적하고 투명성을 확보하기 위한 플랫폼입니다. 12개 기관과 협력하여 2,000여 개 데이터셋의 출처 정보를 제공하며, 개발자들이 데이터의 구성과 계보를 쉽게 파악할 수 있게 돕습니다.",
]

# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

# 데이터프레임 출력
df.head()


,question,answer
0,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.
1,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.
2,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...


In [29]:
pip install --upgrade datasets huggingface_hub


Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
from datasets import load_dataset, Dataset
import os

# huggingface Dataset에서 repo_id로 데이터셋 다운로드
dataset = load_dataset(
    "teddylee777/rag-eval-mini",  # 데이터셋 이름
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # private 데이터인 경우 필요합니다.
)

# 데이터셋에서 split 기준으로 조회
huggingface_df = dataset["korean_v1"].to_pandas()
huggingface_df.head()


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/datasets/teddylee777/rag-eval-mini/resolve/main/README.md (Request ID: Root=1-67ebd46a-155e17111f96a06c4b3e15e4;1d269b69-ff97-4ded-a850-c817bffc5d08)

Invalid credentials in Authorization header